## Lesson 26 - Social Graph with TENET v2.0





### Table of Contents

* [TENET天能](#tenet)
* [Create Keyword Node](#create_keyword)
* [Create More FB Post Node](#create_fb_post_node)
* [Create More FB User Node](#create_fb_user_node)
* [Relationship of MFB Post and User](#relationship_fb_post_user)
* [Relationship of MFB Post keywords and post](#relationship_fb_keyword_post)
* [Relationship of MFB Post hashtag and post](#relationship_fb_hashtag_post)
* [Create More News Post Node](#create_news_node)
* [Create More News User Node](#create_news_user)
* [Relationship of MNews Post and User](#relationship_news_post_user)
* [Relationship of MNews Post keywords and post](#relationship_news_keyword_post)
* [Relationship of MNews Post hashtag and post](#relationship_news_hashtag_post)
* [Create More Forum Post Node](#create_forum_post_node)
* [Create More Forum User Node](#create_forum_user_node)
* [Relationship of MForum Post and User](#relationship_forum_post_user)
* [Relationship of MForum Post keywords and post](#relationship_forum_keyword_post)
* [Relationship of MForum Post hashtag and post](#relationship_forum_hashtag_post)
* [Create More PTT Post Node](#create_ptt_post_node)
* [Create More PTT User Node](#create_ptt_user_node)
* [Relationship of MPtt Post and User](#relationship_ptt_post_user)
* [Relationship of MPtt Post keywords and post](#relationship_ptt_keyword_user)
* [Relationship of MPtt Post hashtag and post](#relationship_ptt_hashtag_user)
* [Import to Neo4j](#import_neo4j)




<a id="tenet"></a>
## TENET天能

- 上映日期：2020-08-27
- 期待度 99% 網友想看 滿意度 4.1 綜合評分
- 資料區間：上映前一周2020-08-17 ~ 上映後一個月2020-09-27
- 資料來源：Facebook, Forum, News, PTT

In [1]:
import csv
import json
import time
import requests
import datetime
import pandas as pd
from datetime import (datetime as dt, timedelta as td)
import numpy as np
from urllib.parse import quote, unquote

pd.options.mode.chained_assignment = None

In [2]:
def extract_hashtag(content):
    hashtags = []
    if type(content)==float:
        return ""
    if '#' in content:
        for h in content.split('#')[1:]:
            hashtags.append('#'+h.split(' ')[0].replace('\n','').replace('、','').strip())
    return "/".join(hashtags) if hashtags else ""

def extract_mention(content):
    mentions = []
    if type(content)==float:
        return ""
    if '@' in content:
        for h in content.split('@')[1:]:
            mentions.append('@'+h.split(' ')[0].replace('\n','').replace('、','').strip())
    return "/".join(mentions) if mentions else ""

def get_keyword_id(keyword):
    return hashtag_keyword_map[str(keyword).strip()]

def add_quote(keyword):
    return '"'+str(keyword).replace('\n', ' ').replace('\r', ' ').strip()+'"'

def replace_double_quote(s):
    return str(s).strip().replace('"',"'")

def if_in_list(item, target_list):
    if item not in target_list:
        return 0
    return 1

In [3]:
import jieba
import jieba.analyse

jieba.initialize()
jieba.set_dictionary("./dict/dict.big.txt")
jieba.load_userdict('./dict/mydic.txt')

wantWordList = set()
with open('./dict/mydic.txt', 'r', encoding="utf8") as file:
    wantWordList = file.readlines()
    wantWordList = [wantword.strip('\n').replace('\ufeff','').strip().split(' ')[0] for wantword in wantWordList]

stopwords = set()
with open('./dict/stopwords.txt', 'r', encoding="utf8") as file:
    stopwords = file.readlines()
    stopwords = [stopword.strip('\n').strip() for stopword in stopwords]
except_file = open("./dict/hippo_exception_word.txt", encoding='utf-8')
exception = except_file.read().split(',')
exception.append(" ")

punct = set(u''':!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
punct |= set(exception)
punct = list(punct)
stopwords = set(stopwords+punct)

names = set()
with open('./dict/name.txt', 'r', encoding="utf-8") as file:
    names= file.readlines()
    names = [name.strip('\n').strip() for name in names]

def segmentWord(text):
    words = [word for word in jieba.cut(text, cut_all=False) if len(word.strip())>1 and (word not in stopwords)  ]
    return "/".join(set(words))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\princ\AppData\Local\Temp\jieba.cache
Loading model cost 0.863 seconds.
Prefix dict has been built succesfully.
Building prefix dict from D:\Programming\Python\Neo4j\課程\dict\dict.big.txt ...
Dumping model to file cache C:\Users\princ\AppData\Local\Temp\jieba.u2119b7327cb8dafa65534ff0e37256b6.cache
Loading model cost 1.623 seconds.
Prefix dict has been built succesfully.


In [4]:
fileName = "天能"
df_more_fb = pd.read_excel(f"./data/excel/{fileName}.xlsx", sheet_name='Facebook', dtype={'content':str})
df_more_news = pd.read_excel(f"./data/excel/{fileName}.xlsx", sheet_name='News', dtype={'content':str})
df_more_ptt = pd.read_excel(f"./data/excel/{fileName}.xlsx", sheet_name='PTT', dtype={'content':str})
df_more_forum = pd.read_excel(f"./data/excel/{fileName}.xlsx", sheet_name='Forum', dtype={'content':str})
if 'dataId' in df_more_fb:
    del df_more_fb['dataId']
if 'dataId' in df_more_news:
    del df_more_news['dataId']
if 'dataId' in df_more_ptt:
    del df_more_ptt['dataId']
if 'dataId' in df_more_forum:
    del df_more_forum['dataId']
df_more_fb.fillna('', inplace=True)
df_more_news.fillna('', inplace=True)
df_more_ptt.fillna('', inplace=True)
df_more_forum.fillna('', inplace=True)

<a id="create_keyword"></a>
## Create Keyword Node

In [5]:
df_more_fb['keywords'] = list(map(segmentWord, df_more_fb['content']))
df_more_news['keywords'] = list(map(segmentWord, df_more_news['content']))
df_more_ptt['keywords'] = list(map(segmentWord, df_more_ptt['content']))
df_more_forum['keywords'] = list(map(segmentWord, df_more_forum['content']))

In [6]:
df_more_fb['hashtags'] = list(map(extract_hashtag, df_more_fb['content']))
df_more_news['hashtags'] = list(map(extract_hashtag, df_more_news['content']))
df_more_ptt['hashtags'] = list(map(extract_hashtag, df_more_ptt['content']))
df_more_forum['hashtags'] = list(map(extract_hashtag, df_more_forum['content']))

In [7]:
df_more_fb['mention'] = list(map(extract_mention, df_more_fb['content']))
df_more_news['mention'] = list(map(extract_mention, df_more_news['content']))
df_more_ptt['mention'] = list(map(extract_mention, df_more_ptt['content']))
df_more_forum['mention'] = list(map(extract_mention, df_more_forum['content']))

In [8]:
more_fb_keyword_list = list(df_more_fb[df_more_fb['keywords'].str.len() > 0]['keywords'])
more_news_keyword_list = list(df_more_news[df_more_news['keywords'].str.len() > 0]['keywords'])
more_ptt_keyword_list = list(df_more_ptt[df_more_ptt['keywords'].str.len() > 0]['keywords'])
more_forum_keyword_list = list(df_more_forum[df_more_forum['keywords'].str.len() > 0]['keywords'])

In [9]:
more_fb_hashtags_list = list(df_more_fb[df_more_fb['hashtags'].str.len() > 0]['hashtags'])
more_news_hashtags_list = list(df_more_news[df_more_news['hashtags'].str.len() > 0]['hashtags'])
more_ptt_hashtags_list = list(df_more_ptt[df_more_ptt['hashtags'].str.len() > 0]['hashtags'])
more_forum_hashtags_list = list(df_more_forum[df_more_forum['hashtags'].str.len() > 0]['hashtags'])

In [10]:
more_fb_mention_list = list(df_more_fb[df_more_fb['mention'].str.len() > 0]['mention'])
more_news_mention_list = list(df_more_news[df_more_news['mention'].str.len() > 0]['mention'])
more_ptt_mention_list = list(df_more_ptt[df_more_ptt['mention'].str.len() > 0]['mention'])
more_forum_mention_list = list(df_more_forum[df_more_forum['mention'].str.len() > 0]['mention'])

In [11]:
more_fb_hashtags_list[3:10]

['#每一張海報都好酷/#爆編拼圖難割捨啊?場次時間：8/25（二）19：00＠美麗華大直影城/#是看IMAX無誤???/#限時一小時_1900前下好離手/#快閃爆好康/#2200與上週節目爆好康同時開獎/#中獎者需要今晚2359前回覆相關資料_並需自付郵資轉帳/#逾時不候/#販賣票券者絕不寬待/#這麼棒的電影節目還需要你的持續鼓勵pressplay.cc/popcornTENET/#Tenet/#天能/#RobertPattinson/#ElizabethDebicki',
 '#TENET天能',
 '#全面啟動/#GQ經典片單/#GQ電影',
 '#這選角「黛安娜王妃是一種精神。她的言語以及事蹟存於許多人心中。能夠在這個打從第一季起便深得我心的影集中飾演她，真的是此生至高無上的榮幸。」/#伊莉莎白戴比基/#王冠》不但確定會推出到第六季，隨著劇中時代越來越接近現代，不少劇迷也期待看見黛安娜王妃能出現其中，而這位王妃也確定將由曾演出《紳士密令》《天能》的影星伊莉莎白戴比基來詮釋。',
 '#台南新天地/#新光影城/#天能/#電影場次全面預購中',
 '#諾蘭導演【/#TENET天能】8月27日(四)上映/#林口威秀/#期待已久的大片',
 '#逆轉時空。導演/#克里斯多夫諾蘭/#約翰大衛華盛頓/#羅伯派汀森/#伊莉莎白戴比姬/#米高肯恩/#肯尼斯布萊納【/#天能】[保謢級]']

In [12]:
df_keyword = pd.DataFrame(more_fb_keyword_list+more_news_keyword_list+more_ptt_keyword_list+more_forum_keyword_list, columns=['keywords'])
bag_keyword_list = []
global_bag_keyword_list = []
keyword_id_map = {}
cnt_keyword = 100_000_000
for index, row in df_keyword.iterrows():
    for keyword in row['keywords'].split('/'):
        if keyword.strip() not in global_bag_keyword_list:
            cnt_keyword = cnt_keyword+1
            cnt_keyword_code = "K"+str(cnt_keyword)
            keyword_id_map[keyword] = cnt_keyword_code
            bag_keyword_list.append({"id":cnt_keyword_code, "keyword":keyword})
            global_bag_keyword_list.append(keyword)

if global_bag_keyword_list:
    # Create keyword nodes
    all_keyword_nodes = pd.DataFrame(bag_keyword_list)
    all_keyword_nodes = all_keyword_nodes.rename(columns={'id':'Keyword:ID'})
    all_keyword_nodes[":LABEL"] = "Keywords"
    all_keyword_nodes["keyword"] = all_keyword_nodes["keyword"].apply(lambda x : replace_double_quote(x))
    all_keyword_nodes["keyword"] = all_keyword_nodes["keyword"].apply(lambda x: add_quote(x))
    all_keyword_nodes.to_csv(f'opt/csv/more_tenet/post_keyword_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty keyword nodes
    keyword_id_map = {}
    all_keyword_nodes = pd.DataFrame(columns=['Keyword:ID', 'keyword', ':LABEL'])
    all_keyword_nodes.to_csv(f'opt/csv/more_tenet/post_keyword_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

In [13]:
df_hashtags = pd.DataFrame(more_fb_hashtags_list+more_news_hashtags_list+more_ptt_hashtags_list+more_forum_hashtags_list, columns=['hashtags'])
bag_hashtags_list = []
global_bag_hashtags_list = []
hashtags_id_map = {}
cnt_hashtags = 100_000_000
for index, row in df_hashtags.iterrows():
    for hashtags in row['hashtags'].split('/'):
        if hashtags.strip() not in global_bag_hashtags_list:
            cnt_hashtags = cnt_hashtags+1
            cnt_hashtags_code = "H"+str(cnt_hashtags)
            hashtags_id_map[hashtags] = cnt_hashtags_code
            bag_hashtags_list.append({"id":cnt_hashtags_code, "hashtags":hashtags})
            global_bag_hashtags_list.append(hashtags)

if global_bag_hashtags_list:
    # Create hashtags nodes
    all_hashtags_nodes = pd.DataFrame(bag_hashtags_list)
    all_hashtags_nodes = all_hashtags_nodes.rename(columns={'id':'Hashtag:ID'})
    all_hashtags_nodes[":LABEL"] = "Hashtag"
    all_hashtags_nodes["hashtags"] = all_hashtags_nodes["hashtags"].apply(lambda x : replace_double_quote(x))
    all_hashtags_nodes["hashtags"] = all_hashtags_nodes["hashtags"].apply(lambda x: add_quote(x))
    all_hashtags_nodes.to_csv(f'opt/csv/more_tenet/post_hashtags_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty hashtags nodes
    hashtags_id_map = {}
    all_hashtags_nodes = pd.DataFrame(columns=['Hashtag:ID', 'hashtags', ':LABEL'])
    all_hashtags_nodes.to_csv(f'opt/csv/more_tenet/post_hashtags_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

In [14]:
df_mention = pd.DataFrame(more_fb_mention_list+more_news_mention_list+more_ptt_mention_list+more_forum_mention_list, columns=['mention'])
bag_mention_list = []
global_bag_mention_list = []
mention_id_map = {}
cnt_mention = 100_000_000
for index, row in df_mention.iterrows():
    for mention in row['mention'].split('/'):
        if mention.strip() not in global_bag_mention_list:
            cnt_mention = cnt_mention+1
            cnt_mention_code = "M"+str(cnt_mention)
            mention_id_map[mention] = cnt_mention_code
            bag_mention_list.append({"id":cnt_mention_code, "mention":mention})
            global_bag_mention_list.append(mention)

if global_bag_mention_list:
    # Create mention nodes
    all_mention_nodes = pd.DataFrame(bag_mention_list)
    all_mention_nodes = all_mention_nodes.rename(columns={'id':'Mention:ID'})
    all_mention_nodes[":LABEL"] = "MENTIONS"
    all_mention_nodes["mention"] = all_mention_nodes["mention"].apply(lambda x : replace_double_quote(x))
    all_mention_nodes["mention"] = all_mention_nodes["mention"].apply(lambda x: add_quote(x))
    all_mention_nodes.to_csv(f'opt/csv/more_tenet/post_mention_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty mention nodes
    mention_id_map = {}
    all_mention_nodes = pd.DataFrame(columns=['Mention:ID', 'mention', ':LABEL'])
    all_mention_nodes.to_csv(f'opt/csv/more_tenet/post_mention_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

<a id="create_fb_post_node"></a>
## Create More FB Post Node

In [15]:
df = df_more_fb.copy()
df.columns

Index(['postLink', 'pageId', 'fromId', 'fromName', 'caption', 'description',
       'linkName', 'link', 'picture', 'likes', 'comments', 'shares',
       'totalResponse', 'postType', 'time', 'crawlTime', 'sentiment',
       'reactionLove', 'reactionWow', 'reactionHaha', 'reactionSad',
       'reactionAngry', 'reactions', 'content', 'keyword', 'keywords',
       'hashtags', 'mention'],
      dtype='object')

In [16]:
# Post:ID,text,:LABEL
mfb_post_nodes = df[['postLink','description','fromName','content','link','picture','likes','comments','shares','postType','time','reactionLove','reactionWow','reactionHaha','reactionSad','reactionAngry']]
mfb_post_nodes[":LABEL"] = "MFBPost"
mfb_post_nodes = mfb_post_nodes.rename(columns={'postLink': 'MFBPost:ID'})
mfb_post_nodes = mfb_post_nodes.drop_duplicates(subset='MFBPost:ID', keep="first")

mfb_post_nodes["MFBPost:ID"] = mfb_post_nodes["MFBPost:ID"].apply(lambda x: "MFB_"+str(x))
mfb_post_nodes["description"] = mfb_post_nodes["description"].apply(lambda x : replace_double_quote(x))
mfb_post_nodes["description"] = mfb_post_nodes["description"].apply(lambda x : add_quote(x))
mfb_post_nodes["fromName"] = mfb_post_nodes["fromName"].apply(lambda x : replace_double_quote(x))
mfb_post_nodes["fromName"] = mfb_post_nodes["fromName"].apply(lambda x : add_quote(x))
mfb_post_nodes["content"] = mfb_post_nodes["content"].apply(lambda x:str(x).replace('\n',' ').strip())
mfb_post_nodes["content"] = mfb_post_nodes["content"].apply(lambda x : replace_double_quote(x))
mfb_post_nodes["content"] = mfb_post_nodes["content"].apply(lambda x : add_quote(x))
mfb_post_nodes.to_csv('opt/csv/more_tenet/mfb_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mfb_post_nodes.shape[0])
mfb_post_nodes.head(1)

1510


,MFBPost:ID,description,fromName,content,link,picture,likes,comments,shares,postType,time,reactionLove,reactionWow,reactionHaha,reactionSad,reactionAngry,:LABEL
0,MFB_https://www.facebook.com/504302659615499/p...,"""#禾浩辰 #活屍 #GetTheHellOut PART2 https://youtu.b...","""爆米花看電影""","""希望X編有 #天能 幫爆米花看電影推上Youtube首頁熱門影片 （這個願望，只要大家多多...",https://popcornmoviestw.pros.si/GTHO02,https://external-nrt1-1.xx.fbcdn.net/safe_imag...,52,29,1,link,2020-08-17 23:21:03,11,1,0,8,0,MFBPost


<a id="create_fb_user_node"></a>
## Create More FB User Node

In [17]:
# MFBUser:ID,username,:LABEL

mfb_user_nodes = df[['postLink','pageId','fromId','fromName']]

mfb_user_nodes = mfb_user_nodes.rename(columns={'pageId':'MFBUser:ID', 'fromName': 'username'})
mfb_user_nodes[":LABEL"] = "MFBUser"
mfb_user_nodes = mfb_user_nodes.drop_duplicates(subset='MFBUser:ID', keep="first")

mfb_user_nodes["link"] = mfb_user_nodes["fromId"].apply(lambda x : f"https://www.facebook.com/{x}")
mfb_user_nodes["MFBUser:ID"] = mfb_user_nodes["MFBUser:ID"].apply(lambda x: "MFB_"+str(x))
mfb_user_nodes["username"] = mfb_user_nodes["username"].apply(lambda x : replace_double_quote(x))
mfb_user_nodes["username"] = mfb_user_nodes["username"].apply(lambda x : add_quote(x))
mfb_user_nodes = mfb_user_nodes[["MFBUser:ID","username","fromId","link",":LABEL"]]
mfb_user_nodes.to_csv('opt/csv/more_tenet/mfb_user_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mfb_user_nodes.shape[0])
mfb_user_nodes.head(1)

488


,MFBUser:ID,username,fromId,link,:LABEL
0,MFB_504302659615499,"""爆米花看電影""",5.04303e+14,https://www.facebook.com/504302659615499.0,MFBUser


<a id="relationship_fb_post_user"></a>
## Relationship of MFB Post and User

In [18]:
# :START_ID,text,:END_ID,:TYPE

to_id_list = list(mfb_post_nodes['MFBPost:ID'])

MFB_user_post_relation = df[['pageId','fromName','postLink']]
MFB_user_post_relation[":START_ID"] = MFB_user_post_relation["pageId"].apply(lambda x:"MFB_"+str(x))
MFB_user_post_relation = MFB_user_post_relation.rename(columns={'fromName': 'username',
                                                                'postLink': ':END_ID'})
MFB_user_post_relation[":END_ID"] = MFB_user_post_relation[":END_ID"].apply(lambda x:"MFB_"+str(x))
MFB_user_post_relation["link"] = MFB_user_post_relation["pageId"].apply(lambda x:"MFB_"+str(x))
MFB_user_post_relation["in_list"] = MFB_user_post_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MFB_user_post_relation = MFB_user_post_relation[MFB_user_post_relation['in_list']==1]
del MFB_user_post_relation['in_list']
MFB_user_post_relation[':TYPE'] = 'POSTS'
MFB_user_post_relation = MFB_user_post_relation[[":START_ID","username","pageId","link",":END_ID",":TYPE"]]
MFB_user_post_relation["username"] = MFB_user_post_relation["username"].apply(lambda x : replace_double_quote(x))
MFB_user_post_relation["username"] = MFB_user_post_relation["username"].apply(lambda x : add_quote(x))
MFB_user_post_relation.to_csv('opt/csv/more_tenet/mfb_user_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MFB_user_post_relation.shape[0])
MFB_user_post_relation.sample(1)

1510


,:START_ID,username,pageId,link,:END_ID,:TYPE
1066,MFB_466505160192708,"""Initium Media 端傳媒""",466505160192708,MFB_466505160192708,MFB_https://www.facebook.com/466505160192708/p...,POSTS


<a id="relationship_fb_keyword_post"></a>
## Relationship of MFB Post keywords and post

In [19]:
to_id_list = list(mfb_post_nodes['MFBPost:ID'])
MFB_keyword_list = []
for index, row in df_more_fb[df_more_fb['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').strip().split('/'):
        MFB_keyword_list.append({"keyword":keyword, "postId":row['postLink']})

MFB_keyword_nodes = pd.DataFrame(MFB_keyword_list)
MFB_keyword_relation = MFB_keyword_nodes[['keyword','postId']]
MFB_keyword_relation = MFB_keyword_relation.rename(columns={'keyword': 'keyword',
                                                            'postId': ':END_ID'})
MFB_keyword_relation[":START_ID"] =  MFB_keyword_relation["keyword"].map(keyword_id_map)
MFB_keyword_relation[":END_ID"] = MFB_keyword_relation[":END_ID"].apply(lambda x:"MFB_"+str(x))
MFB_keyword_relation['in_list'] = MFB_keyword_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MFB_keyword_relation = MFB_keyword_relation[MFB_keyword_relation['in_list']==1]
del MFB_keyword_relation['in_list']
MFB_keyword_relation[':TYPE'] = 'RELATED_TO'
MFB_keyword_relation["keyword"] = MFB_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
MFB_keyword_relation["keyword"] = MFB_keyword_relation["keyword"].apply(lambda x : add_quote(x))
MFB_keyword_relation = MFB_keyword_relation[[":START_ID","keyword",":END_ID",":TYPE"]]
MFB_keyword_relation.to_csv('opt/csv/more_tenet/mfb_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MFB_keyword_relation.shape[0])
MFB_keyword_relation.sample()

73563


,:START_ID,keyword,:END_ID,:TYPE
34922,K100002758,"""完美""",MFB_https://www.facebook.com/483575261691223/p...,RELATED_TO


<a id="relationship_fb_hashtag_post"></a>
## Relationship of MFB Post hashtag and post

In [20]:
to_id_list = list(mfb_post_nodes['MFBPost:ID'])
MFB_hashtag_list = []
for index, row in df_more_fb[df_more_fb['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').strip().split('/'):
        MFB_hashtag_list.append({"hashtag":hashtag, "postId":row['postLink']})

if len(MFB_hashtag_list)>0:
    MFB_hashtag_nodes = pd.DataFrame(MFB_hashtag_list)
    MFB_hashtag_relation = MFB_hashtag_nodes[['hashtag','postId']]
    MFB_hashtag_relation = MFB_hashtag_relation.rename(columns={'hashtag': 'hashtag',
                                                                'postId': ':END_ID'})
    MFB_hashtag_relation[":START_ID"] =  MFB_hashtag_relation["hashtag"].map(hashtags_id_map)
    MFB_hashtag_relation[":END_ID"] = MFB_hashtag_relation[":END_ID"].apply(lambda x:"MFB_"+str(x))
    MFB_hashtag_relation['in_list'] = MFB_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
    MFB_hashtag_relation = MFB_hashtag_relation[MFB_hashtag_relation['in_list']==1]
    del MFB_hashtag_relation['in_list']
    MFB_hashtag_relation[':TYPE'] = 'TAGS'
    MFB_hashtag_relation["hashtag"] = MFB_hashtag_relation["hashtag"].apply(lambda x : replace_double_quote(x))
    MFB_hashtag_relation["hashtag"] = MFB_hashtag_relation["hashtag"].apply(lambda x : add_quote(x))
    MFB_hashtag_relation = MFB_hashtag_relation[[":START_ID","hashtag",":END_ID",":TYPE"]]
    MFB_hashtag_relation.to_csv('opt/csv/more_tenet/mfb_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
    print(MFB_hashtag_relation.shape[0])
else:
    MFB_hashtag_relation = pd.DataFrame(columns=[":START_ID","hashtag",":END_ID",":TYPE"])
    MFB_hashtag_relation.to_csv('opt/csv/more_tenet/mnews_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
MFB_hashtag_relation.head(1)

4661


,:START_ID,hashtag,:END_ID,:TYPE
0,H100000001,"""#天能""",MFB_https://www.facebook.com/504302659615499/p...,TAGS


<a id="create_news_node"></a>
## Create More News Post Node

In [21]:
df = df_more_news.copy()
df.columns

Index(['sitename', 'type', 'url', 'title', 'content', 'time', 'crawlTime',
       'sentiment', 'author', 'keyword', 'keywords', 'hashtags', 'mention'],
      dtype='object')

In [22]:
df.sample()

,sitename,type,url,title,content,time,crawlTime,sentiment,author,keyword,keywords,hashtags,mention
491,match生活網,即時新聞,http://m.match.net.tw/pc/news/entertainment/20...,諾蘭經典之作《星際效應》 下月8日重返大銀幕,導演克里斯多夫諾蘭執導的新作《天能》在台上映以來掀起熱烈討論，觀影人次也突破百萬，繼續保持票...,2020-09-15 23:27:00,2020-09-16 00:21:05,positive,match生活網,"天能""AND""電影",討論/2014/Top/星際/遙遠/西卡/視覺/計劃/動搖/造成/250/主要演員/評分/執...,,


In [23]:
# Post:ID,text,:LABEL
mnews_post_nodes = df[['url','sitename','title','content','type','author','time']]
mnews_post_nodes[":LABEL"] = "MNewsPost"
mnews_post_nodes = mnews_post_nodes.rename(columns={'url': 'MNewsPost:ID'})
mnews_post_nodes = mnews_post_nodes.drop_duplicates(subset='MNewsPost:ID', keep="first")

mnews_post_nodes["MNewsPost:ID"] = mnews_post_nodes["MNewsPost:ID"].apply(lambda x: "MNews_"+str(x))
mnews_post_nodes["sitename"] = mnews_post_nodes["sitename"].apply(lambda x : replace_double_quote(x))
mnews_post_nodes["sitename"] = mnews_post_nodes["sitename"].apply(lambda x : add_quote(x))
mnews_post_nodes["type"] = mnews_post_nodes["type"].apply(lambda x : replace_double_quote(x))
mnews_post_nodes["type"] = mnews_post_nodes["type"].apply(lambda x : add_quote(x))
mnews_post_nodes["title"] = mnews_post_nodes["title"].apply(lambda x : replace_double_quote(x))
mnews_post_nodes["title"] = mnews_post_nodes["title"].apply(lambda x : add_quote(x))
mnews_post_nodes["author"] = mnews_post_nodes["author"].apply(lambda x : replace_double_quote(x))
mnews_post_nodes["author"] = mnews_post_nodes["author"].apply(lambda x : add_quote(x))
mnews_post_nodes["content"] = mnews_post_nodes["content"].apply(lambda x:str(x).replace('\n',' ').strip())
mnews_post_nodes["content"] = mnews_post_nodes["content"].apply(lambda x : replace_double_quote(x))
mnews_post_nodes["content"] = mnews_post_nodes["content"].apply(lambda x : add_quote(x))
mnews_post_nodes.to_csv('opt/csv/more_tenet/mnews_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mnews_post_nodes.shape[0])
mnews_post_nodes.head(1)

943


,MNewsPost:ID,sitename,title,content,type,author,time,:LABEL
0,MNews_https://star.ettoday.net/news/1787084,"""ETtoday""","""李芷婷零下10度看展「老神在在」 曝台灣賞雪秘境！""","""記者翁子涵／台北報導 李芷婷日前推出首張專輯《美麗舊世界》，原本今年還計畫去日本或韓國旅遊...","""影劇""","""ETtoday""",2020-08-17 22:11:00,MNewsPost


<a id="create_news_user"></a>
## Create More News User Node

In [24]:
# MNewsUser:ID,username,:LABEL

mnews_user_nodes = df[['sitename','author','type']]

mnews_user_nodes = mnews_user_nodes.rename(columns={'sitename':'MNewsUser:ID', 'author': 'username'})
mnews_user_nodes[":LABEL"] = "MNewsUser"
mnews_user_nodes = mnews_user_nodes.drop_duplicates(subset='MNewsUser:ID', keep="first")

mnews_user_nodes["MNewsUser:ID"] = mnews_user_nodes["MNewsUser:ID"].apply(lambda x: "MNews_"+str(x))
mnews_user_nodes["username"] = mnews_user_nodes["username"].apply(lambda x : replace_double_quote(x))
mnews_user_nodes["username"] = mnews_user_nodes["username"].apply(lambda x : add_quote(x))
mnews_user_nodes["type"] = mnews_user_nodes["type"].apply(lambda x : replace_double_quote(x))
mnews_user_nodes["type"] = mnews_user_nodes["type"].apply(lambda x : add_quote(x))
mnews_user_nodes = mnews_user_nodes[["MNewsUser:ID","username","type",":LABEL"]]
mnews_user_nodes.to_csv('opt/csv/more_tenet/mnews_user_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mnews_user_nodes.shape[0])
mnews_user_nodes.head(1)

61


,MNewsUser:ID,username,type,:LABEL
0,MNews_ETtoday,"""ETtoday""","""影劇""",MNewsUser


<a id="relationship_news_post_user"></a>
## Relationship of MNews Post and User

In [25]:
# :START_ID,text,:END_ID,:TYPE

to_id_list = list(mnews_post_nodes['MNewsPost:ID'])

MNews_user_post_relation = df[['sitename','author','url']]
MNews_user_post_relation[":START_ID"] = MNews_user_post_relation["sitename"].apply(lambda x:"MNews_"+str(x))
MNews_user_post_relation = MNews_user_post_relation.rename(columns={'author':'username', 'url':':END_ID'})
MNews_user_post_relation[":END_ID"] = MNews_user_post_relation[":END_ID"].apply(lambda x:"MNews_"+str(x))
MNews_user_post_relation["in_list"] = MNews_user_post_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MNews_user_post_relation = MNews_user_post_relation[MNews_user_post_relation['in_list']==1]
del MNews_user_post_relation['in_list']
MNews_user_post_relation[':TYPE'] = 'POSTS'
MNews_user_post_relation = MNews_user_post_relation[[":START_ID","username",":END_ID",":TYPE"]]
MNews_user_post_relation["username"] = MNews_user_post_relation["username"].apply(lambda x : replace_double_quote(x))
MNews_user_post_relation["username"] = MNews_user_post_relation["username"].apply(lambda x : add_quote(x))
MNews_user_post_relation.to_csv('opt/csv/more_tenet/mnews_user_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MNews_user_post_relation.shape[0])
MNews_user_post_relation.sample(1)

943


,:START_ID,username,:END_ID,:TYPE
262,MNews_妞新聞,"""妞新聞""",MNews_https://www.niusnews.com/=P2qe5sy2,POSTS


<a id="relationship_news_keyword_post"></a>
## Relationship of MNews Post keywords and post

In [26]:
to_id_list = list(mnews_post_nodes['MNewsPost:ID'])
MNews_keyword_list = []
for index, row in df_more_news[df_more_news['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').strip().split('/'):
        MNews_keyword_list.append({"keyword":keyword, "postId":row['url']})

MNews_keyword_nodes = pd.DataFrame(MNews_keyword_list)
MNews_keyword_relation = MNews_keyword_nodes[['keyword','postId']]
MNews_keyword_relation = MNews_keyword_relation.rename(columns={'keyword': 'keyword',
                                                            'postId': ':END_ID'})
MNews_keyword_relation[":START_ID"] =  MNews_keyword_relation["keyword"].map(keyword_id_map)
MNews_keyword_relation[":END_ID"] = MNews_keyword_relation[":END_ID"].apply(lambda x:"MNews_"+str(x))
MNews_keyword_relation['in_list'] = MNews_keyword_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MNews_keyword_relation = MNews_keyword_relation[MNews_keyword_relation['in_list']==1]
del MNews_keyword_relation['in_list']
MNews_keyword_relation[':TYPE'] = 'RELATED_TO'
MNews_keyword_relation["keyword"] = MNews_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
MNews_keyword_relation["keyword"] = MNews_keyword_relation["keyword"].apply(lambda x : add_quote(x))
MNews_keyword_relation = MNews_keyword_relation[[":START_ID","keyword",":END_ID",":TYPE"]]
MNews_keyword_relation.to_csv('opt/csv/more_tenet/mnews_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MNews_keyword_relation.shape[0])
MNews_keyword_relation.sample()

216929


,:START_ID,keyword,:END_ID,:TYPE
160091,K100005137,"""改編自""",MNews_https://today.line.me/tw/v2/article/1eGlzp,RELATED_TO


<a id="relationship_news_hashtag_post"></a>
## Relationship of MNews Post hashtag and post

In [27]:
to_id_list = list(mnews_post_nodes['MNewsPost:ID'])
MNews_hashtag_list = []
for index, row in df_more_news[df_more_news['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').strip().split('/'):
        MNews_hashtag_list.append({"hashtag":hashtag, "postId":row['url']})

if len(MNews_hashtag_list)>0:
    MNews_hashtag_nodes = pd.DataFrame(MNews_hashtag_list)
    MNews_hashtag_relation = MNews_hashtag_nodes[['hashtag','postId']]
    MNews_hashtag_relation = MNews_hashtag_relation.rename(columns={'hashtag': 'hashtag',
                                                                'postId': ':END_ID'})
    MNews_hashtag_relation[":START_ID"] =  MNews_hashtag_relation["hashtag"].map(hashtags_id_map)
    MNews_hashtag_relation[":END_ID"] = MNews_hashtag_relation[":END_ID"].apply(lambda x:"MNews_"+str(x))
    MNews_hashtag_relation['in_list'] = MNews_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
    MNews_hashtag_relation = MNews_hashtag_relation[MNews_hashtag_relation['in_list']==1]
    del MNews_hashtag_relation['in_list']
    MNews_hashtag_relation[':TYPE'] = 'RELATED_TO'
    MNews_hashtag_relation["hashtag"] = MNews_hashtag_relation["hashtag"].apply(lambda x : replace_double_quote(x))
    MNews_hashtag_relation["hashtag"] = MNews_hashtag_relation["hashtag"].apply(lambda x : add_quote(x))
    MNews_hashtag_relation = MNews_hashtag_relation[[":START_ID","hashtag",":END_ID",":TYPE"]]
    MNews_hashtag_relation.to_csv('opt/csv/more_tenet/mnews_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
    print(MNews_hashtag_relation.shape[0])
else:
    MNews_hashtag_relation = pd.DataFrame(columns=[":START_ID","hashtag",":END_ID",":TYPE"])
    MNews_hashtag_relation.to_csv('opt/csv/more_tenet/mnews_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
MNews_hashtag_relation.head(1)

189


,:START_ID,hashtag,:END_ID,:TYPE
0,H100000438,"""#""",MNews_https://www.chinatimes.com/newspapers/20...,RELATED_TO


<a id="create_forum_post_node"></a>
## Create More Forum Post Node

In [28]:
df = df_more_forum.copy()
df.columns

Index(['forum', 'channel', 'url', 'title', 'content', 'replyCount', 'time',
       'crawlTime', 'author', 'isReply', 'sentiment', 'postNumInThread',
       'keyword', 'keywords', 'hashtags', 'mention'],
      dtype='object')

In [29]:
df.sample()

,forum,channel,url,title,content,replyCount,time,crawlTime,author,isReply,sentiment,postNumInThread,keyword,keywords,hashtags,mention
248,Mobile01,腕錶綜合討論,https://www.mobile01.com/topicdetail.php?f=376...,「Hamilton Khaki Pilot」漢米爾頓卡其飛行計時錶 - 開別人箱開箱文,▲\n回顧這兩年，開箱類型總是離不開電競周邊與3C產品\n即便期間自己也買了新錶紀念人生里程...,1,2020-12-14 22:35:00,2020-12-15 06:34:49,Mr.Lazy,False,neutral,1,"天能""AND""電影",絕地/標誌/成名/搞笑/功能/131/小勞勃道/乾淨/美感/當作/又換/外太空/並快/扣式/...,#10引言分享,


In [30]:
# Post:ID,text,:LABEL
mforum_post_nodes = df[['url','forum','channel','title','content','author','replyCount','isReply','postNumInThread','time']]
mforum_post_nodes[":LABEL"] = "MForumPost"
mforum_post_nodes = mforum_post_nodes.rename(columns={'url': 'MForumPost:ID'})
mforum_post_nodes = mforum_post_nodes.drop_duplicates(subset='MForumPost:ID', keep="first")

mforum_post_nodes["MForumPost:ID"] = mforum_post_nodes["MForumPost:ID"].apply(lambda x: "MForum_"+str(x))
mforum_post_nodes["forum"] = mforum_post_nodes["forum"].apply(lambda x : replace_double_quote(x))
mforum_post_nodes["forum"] = mforum_post_nodes["forum"].apply(lambda x : add_quote(x))
mforum_post_nodes["channel"] = mforum_post_nodes["channel"].apply(lambda x : replace_double_quote(x))
mforum_post_nodes["channel"] = mforum_post_nodes["channel"].apply(lambda x : add_quote(x))
mforum_post_nodes["title"] = mforum_post_nodes["title"].apply(lambda x : replace_double_quote(x))
mforum_post_nodes["title"] = mforum_post_nodes["title"].apply(lambda x : add_quote(x))
mforum_post_nodes["author"] = mforum_post_nodes["author"].apply(lambda x : replace_double_quote(x))
mforum_post_nodes["author"] = mforum_post_nodes["author"].apply(lambda x : add_quote(x))
mforum_post_nodes["content"] = mforum_post_nodes["content"].apply(lambda x:str(x).replace('\n',' ').strip())
mforum_post_nodes["content"] = mforum_post_nodes["content"].apply(lambda x : replace_double_quote(x))
mforum_post_nodes["content"] = mforum_post_nodes["content"].apply(lambda x : add_quote(x))
mforum_post_nodes.to_csv('opt/csv/more_tenet/mforum_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mforum_post_nodes.shape[0])
mforum_post_nodes.head(1)

139


,MForumPost:ID,forum,channel,title,content,author,replyCount,isReply,postNumInThread,time,:LABEL
0,MForum_https://www.mobile01.com/topicdetail.ph...,"""Mobile01""","""小惡魔影城""","""諾蘭最新神作提前燒腦！ 小惡魔邀你前往《天能》全台試映會！""","""諾蘭最新神作天能 必看 悶太久 好就沒去電影院了""","""crazyyo""",,True,379,2020-08-18 23:57:00,MForumPost


<a id="create_forum_user_node"></a>
## Create More Forum User Node

In [31]:
# MForumUser:ID,username,:LABEL

mforum_user_nodes = df[['forum','channel','author']]

mforum_user_nodes = mforum_user_nodes.rename(columns={'forum':'MForumUser:ID', 'author': 'username'})
mforum_user_nodes[":LABEL"] = "MForumUser"
mforum_user_nodes = mforum_user_nodes.drop_duplicates(subset='MForumUser:ID', keep="first")

mforum_user_nodes["MForumUser:ID"] = mforum_user_nodes["MForumUser:ID"].apply(lambda x: "MForum_"+str(x))
mforum_user_nodes["username"] = mforum_user_nodes["username"].apply(lambda x : replace_double_quote(x))
mforum_user_nodes["username"] = mforum_user_nodes["username"].apply(lambda x : add_quote(x))
mforum_user_nodes["channel"] = mforum_user_nodes["channel"].apply(lambda x : replace_double_quote(x))
mforum_user_nodes["channel"] = mforum_user_nodes["channel"].apply(lambda x : add_quote(x))
mforum_user_nodes = mforum_user_nodes[["MForumUser:ID","username","channel",":LABEL"]]
mforum_user_nodes.to_csv('opt/csv/more_tenet/mforum_user_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mforum_user_nodes.shape[0])
mforum_user_nodes.head(1)

3


,MForumUser:ID,username,channel,:LABEL
0,MForum_Mobile01,"""crazyyo""","""小惡魔影城""",MForumUser


<a id="relationship_forum_post_user"></a>
## Relationship of MForum Post and User

In [32]:
# :START_ID,text,:END_ID,:TYPE

to_id_list = list(mforum_post_nodes['MForumPost:ID'])

MForum_user_post_relation = df[['forum','author','url']]
MForum_user_post_relation[":START_ID"] = MForum_user_post_relation["forum"].apply(lambda x:"MForum_"+str(x))
MForum_user_post_relation = MForum_user_post_relation.rename(columns={'author':'username', 'url':':END_ID'})
MForum_user_post_relation[":END_ID"] = MForum_user_post_relation[":END_ID"].apply(lambda x:"MForum_"+str(x))
MForum_user_post_relation["in_list"] = MForum_user_post_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MForum_user_post_relation = MForum_user_post_relation[MForum_user_post_relation['in_list']==1]
del MForum_user_post_relation['in_list']
MForum_user_post_relation[':TYPE'] = 'POSTS'
MForum_user_post_relation = MForum_user_post_relation[[":START_ID","username",":END_ID",":TYPE"]]
MForum_user_post_relation["username"] = MForum_user_post_relation["username"].apply(lambda x : replace_double_quote(x))
MForum_user_post_relation["username"] = MForum_user_post_relation["username"].apply(lambda x : add_quote(x))
MForum_user_post_relation.to_csv('opt/csv/more_tenet/mforum_user_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MForum_user_post_relation.shape[0])
MForum_user_post_relation.sample(1)

249


,:START_ID,username,:END_ID,:TYPE
121,MForum_Mobile01,"""soziwow""",MForum_https://www.mobile01.com/topicdetail.ph...,POSTS


<a id="relationship_forum_keyword_post"></a>
## Relationship of MForum Post keywords and post

In [33]:
to_id_list = list(mforum_post_nodes['MForumPost:ID'])
MForum_keyword_list = []
for index, row in df_more_forum[df_more_forum['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').strip().split('/'):
        MForum_keyword_list.append({"keyword":keyword, "postId":row['url']})

MForum_keyword_nodes = pd.DataFrame(MForum_keyword_list)
MForum_keyword_relation = MForum_keyword_nodes[['keyword','postId']]
MForum_keyword_relation = MForum_keyword_relation.rename(columns={'keyword': 'keyword',
                                                            'postId': ':END_ID'})
MForum_keyword_relation[":START_ID"] =  MForum_keyword_relation["keyword"].map(keyword_id_map)
MForum_keyword_relation[":END_ID"] = MForum_keyword_relation[":END_ID"].apply(lambda x:"MForum_"+str(x))
MForum_keyword_relation['in_list'] = MForum_keyword_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MForum_keyword_relation = MForum_keyword_relation[MForum_keyword_relation['in_list']==1]
del MForum_keyword_relation['in_list']
MForum_keyword_relation[':TYPE'] = 'RELATED_TO'
MForum_keyword_relation["keyword"] = MForum_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
MForum_keyword_relation["keyword"] = MForum_keyword_relation["keyword"].apply(lambda x : add_quote(x))
MForum_keyword_relation = MForum_keyword_relation[[":START_ID","keyword",":END_ID",":TYPE"]]
MForum_keyword_relation.to_csv('opt/csv/more_tenet/mforum_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MForum_keyword_relation.shape[0])
MForum_keyword_relation.sample()

22540


,:START_ID,keyword,:END_ID,:TYPE
21381,K100001854,"""蝙蝠俠""",MForum_https://www.mobile01.com/topicdetail.ph...,RELATED_TO


<a id="relationship_forum_hashtag_post"></a>
## Relationship of MForum Post hashtag and post

In [34]:
to_id_list = list(mforum_post_nodes['MForumPost:ID'])
MForum_hashtag_list = []
for index, row in df_more_forum[df_more_forum['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').strip().split('/'):
        MForum_hashtag_list.append({"hashtag":hashtag, "postId":row['url']})

if len(MForum_hashtag_list)>0:
    MForum_hashtag_nodes = pd.DataFrame(MForum_hashtag_list)
    MForum_hashtag_relation = MForum_hashtag_nodes[['hashtag','postId']]
    MForum_hashtag_relation = MForum_hashtag_relation.rename(columns={'hashtag': 'hashtag',
                                                                'postId': ':END_ID'})
    MForum_hashtag_relation[":START_ID"] =  MForum_hashtag_relation["hashtag"].map(hashtags_id_map)
    MForum_hashtag_relation[":END_ID"] = MForum_hashtag_relation[":END_ID"].apply(lambda x:"MForum_"+str(x))
    MForum_hashtag_relation['in_list'] = MForum_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
    MForum_hashtag_relation = MForum_hashtag_relation[MForum_hashtag_relation['in_list']==1]
    del MForum_hashtag_relation['in_list']
    MForum_hashtag_relation[':TYPE'] = 'RELATED_TO'
    MForum_hashtag_relation["hashtag"] = MForum_hashtag_relation["hashtag"].apply(lambda x : replace_double_quote(x))
    MForum_hashtag_relation["hashtag"] = MForum_hashtag_relation["hashtag"].apply(lambda x : add_quote(x))
    MForum_hashtag_relation = MForum_hashtag_relation[[":START_ID","hashtag",":END_ID",":TYPE"]]
    MForum_hashtag_relation.to_csv('opt/csv/more_tenet/mforum_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
    print(MForum_hashtag_relation.shape[0])
else:
    MForum_hashtag_relation = pd.DataFrame(columns=[":START_ID","hashtag",":END_ID",":TYPE"])
    MForum_hashtag_relation.to_csv('opt/csv/more_tenet/mforum_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
MForum_hashtag_relation.head(1)

38


,:START_ID,hashtag,:END_ID,:TYPE
0,H100002798,"""#1""",MForum_https://www.mobile01.com/topicdetail.ph...,RELATED_TO


<a id="create_ptt_post_node"></a>
## Create More PTT Post Node

In [35]:
df = df_more_ptt.copy()
df.columns

Index(['board', 'url', 'author', 'title', 'content', 'replyCount', 'time',
       'crawlTime', 'isReply', 'sentiment', 'bull', 'comment', 'bear',
       'keyword', 'keywords', 'hashtags', 'mention'],
      dtype='object')

In [36]:
df.sample()

,board,url,author,title,content,replyCount,time,crawlTime,isReply,sentiment,bull,comment,bear,keyword,keywords,hashtags,mention
341,Gossiping,https://www.ptt.cc/bbs/Gossiping/M.1598753691....,oblivionion,[問卦] 高雄人在電影院椅子撒尿？,如題\n爆x公社網友爆料\n在高雄的電影院 看天能\n坐到椅子都是滿滿的尿\n全身褲子濕透\...,88,2020-08-30 10:14:45,2020-08-30 23:47:57,False,neutral,55,28,5,"天能""AND""電影",電影院/看天能/高雄/網友/濕透/全身/公社/沾滿/滿滿的/褲子/爆料/椅子,,


In [37]:
# Post:ID,text,:LABEL
mptt_post_nodes = df[['url','board','title','content','author','replyCount','bull','comment','bear','time']]
mptt_post_nodes[":LABEL"] = "MPttPost"
mptt_post_nodes = mptt_post_nodes.rename(columns={'url': 'MPttPost:ID'})
mptt_post_nodes = mptt_post_nodes.drop_duplicates(subset='MPttPost:ID', keep="first")

mptt_post_nodes["MPttPost:ID"] = mptt_post_nodes["MPttPost:ID"].apply(lambda x: "MPtt_"+str(x))
mptt_post_nodes["title"] = mptt_post_nodes["title"].apply(lambda x : replace_double_quote(x))
mptt_post_nodes["title"] = mptt_post_nodes["title"].apply(lambda x : add_quote(x))
mptt_post_nodes["author"] = mptt_post_nodes["author"].apply(lambda x : replace_double_quote(x))
mptt_post_nodes["author"] = mptt_post_nodes["author"].apply(lambda x : add_quote(x))
mptt_post_nodes["content"] = mptt_post_nodes["content"].apply(lambda x:str(x).replace('\n',' ').strip())
mptt_post_nodes["content"] = mptt_post_nodes["content"].apply(lambda x : replace_double_quote(x))
mptt_post_nodes["content"] = mptt_post_nodes["content"].apply(lambda x : add_quote(x))
mptt_post_nodes.to_csv('opt/csv/more_tenet/mptt_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mptt_post_nodes.shape[0])
mptt_post_nodes.head(1)

1444


,MPttPost:ID,board,title,content,author,replyCount,bull,comment,bear,time,:LABEL
0,MPtt_https://www.ptt.cc/bbs/AllTogether/M.1597...,AllTogether,"""[徵女] 七夕徵個緣分""","""父親節剛過，就被念怎麼還是單身 我也不想阿QQ 就身邊真的沒什麼異性 希望能靠著七夕題材 ...","""yiyuyi""",0,0,0,0,2020-08-17 23:42:59,MPttPost


<a id="create_ptt_user_node"></a>
## Create More PTT User Node

In [38]:
# MPttUser:ID,username,:LABEL

mptt_user_nodes = df[['author']]
mptt_user_nodes['username'] = mptt_user_nodes["author"]
mptt_user_nodes = mptt_user_nodes.rename(columns={'author':'MPttUser:ID'})
mptt_user_nodes[":LABEL"] = "MPttUser"
mptt_user_nodes = mptt_user_nodes.drop_duplicates(subset='MPttUser:ID', keep="first")

mptt_user_nodes["MPttUser:ID"] = mptt_user_nodes["MPttUser:ID"].apply(lambda x: "MPtt_"+str(x))
mptt_user_nodes["username"] = mptt_user_nodes["username"].apply(lambda x : replace_double_quote(x))
mptt_user_nodes["username"] = mptt_user_nodes["username"].apply(lambda x : add_quote(x))
# mptt_user_nodes["MPttUser:ID"] = mptt_user_nodes["MPttUser:ID"].apply(lambda x : replace_double_quote(x))
# mptt_user_nodes["MPttUser:ID"] = mptt_user_nodes["MPttUser:ID"].apply(lambda x : add_quote(x))
mptt_user_nodes = mptt_user_nodes[["MPttUser:ID","username",":LABEL"]]
mptt_user_nodes.to_csv('opt/csv/more_tenet/mptt_user_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(mptt_user_nodes.shape[0])
mptt_user_nodes.head(1)

827


,MPttUser:ID,username,:LABEL
0,MPtt_yiyuyi,"""yiyuyi""",MPttUser


<a id="relationship_ptt_post_user"></a>
## Relationship of MPtt Post and User

In [39]:
# :START_ID,text,:END_ID,:TYPE

to_id_list = list(mptt_post_nodes['MPttPost:ID'])

MPtt_user_post_relation = df[['author','url']]
MPtt_user_post_relation[":START_ID"] = MPtt_user_post_relation["author"].apply(lambda x:"MPtt_"+str(x))
MPtt_user_post_relation = MPtt_user_post_relation.rename(columns={'author':'username', 'url':':END_ID'})
MPtt_user_post_relation[":END_ID"] = MPtt_user_post_relation[":END_ID"].apply(lambda x:"MPtt_"+str(x))
MPtt_user_post_relation["in_list"] = MPtt_user_post_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MPtt_user_post_relation = MPtt_user_post_relation[MPtt_user_post_relation['in_list']==1]
del MPtt_user_post_relation['in_list']
MPtt_user_post_relation[':TYPE'] = 'POSTS'
MPtt_user_post_relation = MPtt_user_post_relation[[":START_ID","username",":END_ID",":TYPE"]]
MPtt_user_post_relation["username"] = MPtt_user_post_relation["username"].apply(lambda x : replace_double_quote(x))
MPtt_user_post_relation["username"] = MPtt_user_post_relation["username"].apply(lambda x : add_quote(x))
MPtt_user_post_relation.to_csv('opt/csv/more_tenet/mptt_user_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MPtt_user_post_relation.shape[0])
MPtt_user_post_relation.sample(1)

1444


,:START_ID,username,:END_ID,:TYPE
1382,MPtt_laramusic,"""laramusic""",MPtt_https://www.ptt.cc/bbs/Drama-Ticket/M.160...,POSTS


<a id="relationship_ptt_keyword_user"></a>
## Relationship of MPtt Post keywords and post

In [40]:
to_id_list = list(mptt_post_nodes['MPttPost:ID'])
MPtt_keyword_list = []
for index, row in df_more_ptt[df_more_ptt['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').strip().split('/'):
        MPtt_keyword_list.append({"keyword":keyword, "postId":row['url']})

MPtt_keyword_nodes = pd.DataFrame(MPtt_keyword_list)
MPtt_keyword_relation = MPtt_keyword_nodes[['keyword','postId']]
MPtt_keyword_relation = MPtt_keyword_relation.rename(columns={'keyword': 'keyword',
                                                            'postId': ':END_ID'})
MPtt_keyword_relation[":START_ID"] =  MPtt_keyword_relation["keyword"].map(keyword_id_map)
MPtt_keyword_relation[":END_ID"] = MPtt_keyword_relation[":END_ID"].apply(lambda x:"MPtt_"+str(x))
MPtt_keyword_relation['in_list'] = MPtt_keyword_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
MPtt_keyword_relation = MPtt_keyword_relation[MPtt_keyword_relation['in_list']==1]
del MPtt_keyword_relation['in_list']
MPtt_keyword_relation[':TYPE'] = 'RELATED_TO'
MPtt_keyword_relation["keyword"] = MPtt_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
MPtt_keyword_relation["keyword"] = MPtt_keyword_relation["keyword"].apply(lambda x : add_quote(x))
MPtt_keyword_relation = MPtt_keyword_relation[[":START_ID","keyword",":END_ID",":TYPE"]]
MPtt_keyword_relation.to_csv('opt/csv/more_tenet/mptt_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(MPtt_keyword_relation.shape[0])
MPtt_keyword_relation.sample()

233684


,:START_ID,keyword,:END_ID,:TYPE
23563,K100046350,"""188""",MPtt_https://www.ptt.cc/bbs/Drama-Ticket/M.159...,RELATED_TO


<a id="relationship_ptt_hashtag_user"></a>
## Relationship of MPtt Post hashtag and post

In [41]:
to_id_list = list(mptt_post_nodes['MPttPost:ID'])
MPtt_hashtag_list = []
for index, row in df_more_ptt[df_more_ptt['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').strip().split('/'):
        MPtt_hashtag_list.append({"hashtag":hashtag, "postId":row['url']})

if len(MPtt_hashtag_list)>0:
    MPtt_hashtag_nodes = pd.DataFrame(MPtt_hashtag_list)
    MPtt_hashtag_relation = MPtt_hashtag_nodes[['hashtag','postId']]
    MPtt_hashtag_relation = MPtt_hashtag_relation.rename(columns={'hashtag': 'hashtag',
                                                                'postId': ':END_ID'})
    MPtt_hashtag_relation[":START_ID"] =  MPtt_hashtag_relation["hashtag"].map(hashtags_id_map)
    MPtt_hashtag_relation[":END_ID"] = MPtt_hashtag_relation[":END_ID"].apply(lambda x:"MPtt_"+str(x))
    MPtt_hashtag_relation['in_list'] = MPtt_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
    MPtt_hashtag_relation = MPtt_hashtag_relation[MPtt_hashtag_relation['in_list']==1]
    del MPtt_hashtag_relation['in_list']
    MPtt_hashtag_relation[':TYPE'] = 'RELATED_TO'
    MPtt_hashtag_relation["hashtag"] = MPtt_hashtag_relation["hashtag"].apply(lambda x : replace_double_quote(x))
    MPtt_hashtag_relation["hashtag"] = MPtt_hashtag_relation["hashtag"].apply(lambda x : add_quote(x))
    MPtt_hashtag_relation = MPtt_hashtag_relation[[":START_ID","hashtag",":END_ID",":TYPE"]]
    MPtt_hashtag_relation.to_csv('opt/csv/more_tenet/mptt_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
    print(MPtt_hashtag_relation.shape[0])
else:
    MPtt_hashtag_relation = pd.DataFrame(columns=[":START_ID","hashtag",":END_ID",":TYPE"])
    MPtt_hashtag_relation.to_csv('opt/csv/more_tenet/mptt_post_hashtag_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
MPtt_hashtag_relation.head(1)

283


,:START_ID,hashtag,:END_ID,:TYPE
0,H100002681,"""#1VEm8SB9""",MPtt_https://www.ptt.cc/bbs/movie/M.1597702829...,RELATED_TO


<a id="import_neo4j"></a>
## Import to Neo4j

```
.\bin\neo4j-admin import --nodes=import/mfb_post_nodes.csv --nodes=import/mfb_user_nodes.csv --nodes=import/mforum_post_nodes.csv --nodes=import/mforum_user_nodes.csv --nodes=import/mnews_post_nodes.csv --nodes=import/mnews_user_nodes.csv --nodes=import/mptt_post_nodes.csv --nodes=import/mptt_user_nodes.csv --nodes=import/post_hashtags_nodes.csv --nodes=import/post_keyword_nodes.csv --nodes=import/post_mention_nodes.csv --relationships=import/mfb_post_hashtag_rel.csv --relationships=import/mfb_post_keyword_rel.csv --relationships=import/mfb_user_post_rel.csv --relationships=import/mforum_post_hashtag_rel.csv --relationships=import/mforum_post_keyword_rel.csv --relationships=import/mforum_user_post_rel.csv --relationships=import/mnews_post_hashtag_rel.csv --relationships=import/mnews_post_keyword_rel.csv --relationships=import/mnews_user_post_rel.csv --relationships=import/mptt_post_hashtag_rel.csv --relationships=import/mptt_post_keyword_rel.csv --relationships=import/mptt_user_post_rel.csv
```

## Homework
- Import data to neo4j.
- Create Mention relationships between user, post.
- Generate Cypher query to see if any connection between each nodes.
